In [1]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Tuple
import logging

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
@dataclass
class StudyParameters:
    """Class to store and validate study parameters."""
    T: int  # Booking horizon
    N: int  # Service horizon
    C: int  # Room capacity
    price_min: float  # Minimum price
    price_max: float  # Maximum price
    alpha: float  # Smoothing parameter for price acceptance
    beta: float  # Smoothing parameter for capacity
    
    def __post_init__(self):
        """Validate parameters after initialization."""
        if self.T <= 0:
            raise ValueError("Booking horizon T must be positive")
        if self.N <= 0:
            raise ValueError("Service horizon N must be positive")
        if self.C <= 0:
            raise ValueError("Capacity C must be positive")
        if self.price_min >= self.price_max:
            raise ValueError("Minimum price must be less than maximum price")
        if self.alpha <= 0 or self.beta <= 0:
            raise ValueError("Smoothing parameters must be positive")

In [8]:
class DataGenerator:
    """Generate data for hotel dynamic pricing study."""
    
    def __init__(self, params: StudyParameters, seed: int = None):
        """
        Initialize the data generator.
        
        Args:
            params: StudyParameters object containing study parameters
            seed: Random seed for reproducibility
        """
        self.params = params
        self.rng = np.random.default_rng(seed)
        self.booking_classes = self._generate_booking_classes()
        logger.info(f"Initialized DataGenerator with {len(self.booking_classes)} booking classes")
    
    def _generate_booking_classes(self) -> List[Tuple[int, int]]:
        """
        Generate booking classes (arrival day, departure day pairs) with a maximum
        length of stay of 7 nights.
        
        Returns:
            List of tuples (arrival_day, departure_day)
        """
        MAX_LOS = 7  # Maximum length of stay
        booking_classes = []
        for arrival in range(1, self.params.N + 1):
            for los in range(1, min(MAX_LOS + 1, self.params.N - arrival + 2)):
                departure = arrival + los - 1
                booking_classes.append((arrival, departure))
        
        logger.info(f"Generated {len(booking_classes)} booking classes with max LOS = {MAX_LOS}")
        return booking_classes
    
    def generate_arrival_probabilities(self, demand_scenario: str = 'base') -> Dict[int, Dict[Tuple[int, int], float]]:
        """
        Generate arrival probabilities incorporating different demand scenarios.
        
        Args:
            demand_scenario: Type of demand pattern to generate:
                           'base': Normal demand pattern
                           'high': Increased arrival probabilities
                           'low': Decreased arrival probabilities
                           'peak': Time-dependent demand spikes
                           'fluctuating': Higher demand variability
        """
        if demand_scenario not in ['base', 'high', 'low', 'peak', 'fluctuating']:
            raise ValueError(f"Invalid demand scenario: {demand_scenario}")

        # Define scenario-specific parameters
        scenario_params = {
            'base': {
                'demand_multiplier': 1.0,
                'variability': 0.2,
                'peak_periods': []
            },
            'high': {
                'demand_multiplier': 1.5,
                'variability': 0.2,
                'peak_periods': []
            },
            'low': {
                'demand_multiplier': 0.5,
                'variability': 0.2,
                'peak_periods': []
            },
            'peak': {
                'demand_multiplier': 1.0,
                'variability': 0.2,
                'peak_periods': [(self.params.T // 2, 2.0)]  # (period, multiplier)
            },
            'fluctuating': {
                'demand_multiplier': 1.0,
                'variability': 0.4,
                'peak_periods': []
            }
        }

        # Get scenario parameters
        scenario = scenario_params[demand_scenario]
        
        # Define day-of-week seasonality factors
        dow_factors = {
            0: 0.8,  # Sunday
            1: 0.5,  # Monday
            2: 0.5,  # Tuesday
            3: 0.5,  # Wednesday
            4: 0.6,  # Thursday
            5: 1.0,  # Friday
            6: 1.0   # Saturday
        }
        
        arrival_probs = {}
        for t in range(1, self.params.T + 1):
            base_probs = []
            for arrival, departure in self.booking_classes:
                # Calculate basic factors
                los = departure - arrival + 1
                days_until_arrival = arrival + (self.params.T - t)
                
                # Calculate day-of-week effect
                stay_days = range(arrival, departure + 1)
                stay_dow_factors = [dow_factors[((d-1) % 7)] for d in stay_days]
                avg_dow_factor = np.mean(stay_dow_factors)
                
                # Calculate base probability
                los_factor = np.exp(-0.2 * (los - 1))
                time_factor = np.exp(-0.1 * days_until_arrival)
                base_prob = los_factor * time_factor * avg_dow_factor
                base_probs.append(base_prob)
            
            # Apply scenario adjustments
            base_probs = np.array(base_probs) * scenario['demand_multiplier']
            
            # Apply peak period effects if any
            for peak_t, peak_multiplier in scenario['peak_periods']:
                distance = abs(t - peak_t)
                if distance <= 2:  # Effect spans 5 periods
                    peak_effect = peak_multiplier * (1 - 0.3 * distance)
                    base_probs *= peak_effect
            
            # Add random variation
            raw_probs = base_probs * (1 + self.rng.uniform(
                -scenario['variability'],
                scenario['variability'],
                len(base_probs)
            ))
            
            # Normalize to ensure sum < 1
            total_prob = np.sum(raw_probs)
            if total_prob > 0:
                scale_factor = 0.95 / max(total_prob, 1)  # Ensure sum is at most 0.95
                scaled_probs = raw_probs * scale_factor
            else:
                scaled_probs = raw_probs
            
            arrival_probs[t] = {
                class_: prob for class_, prob in zip(self.booking_classes, scaled_probs)
            }

        # Log scenario statistics
        daily_arrival_probs = [sum(probs.values()) for probs in arrival_probs.values()]
        logger.info(f"\nDemand Scenario: {demand_scenario}")
        logger.info(f"Average daily arrival probability: {np.mean(daily_arrival_probs):.3f}")
        logger.info(f"Maximum daily arrival probability: {np.max(daily_arrival_probs):.3f}")
        logger.info(f"Minimum daily arrival probability: {np.min(daily_arrival_probs):.3f}")
        
        # Log day-of-week statistics
        dow_probs = {dow: [] for dow in range(7)}
        for t, probs in arrival_probs.items():
            for (arrival, departure), prob in probs.items():
                dow = (arrival - 1) % 7
                dow_probs[dow].append(prob)
        
        logger.info("\nAverage arrival probabilities by day of week:")
        for dow in range(7):
            day_name = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'][dow]
            avg_prob = np.mean(dow_probs[dow]) if dow_probs[dow] else 0
            logger.info(f"{day_name}: {avg_prob:.3f}")
        
        return arrival_probs
    
    def generate_reservation_price_params(self) -> Dict[Tuple[int, int], float]:
        """
        Generate epsilon parameters for the linear reservation price survival function.
        F̄(p) = 1 - ε * p for each booking class.
        
        Returns:
            Dictionary mapping booking class to epsilon parameter
        """
        # Generate epsilon values ensuring F̄(p_max) >= 0
        max_epsilon = 1 / self.params.price_max  # Ensures F̄(p_max) >= 0
        epsilons = {
            class_: self.rng.uniform(0, max_epsilon)
            for class_ in self.booking_classes
        }
        
        logger.info("Generated reservation price parameters")
        return epsilons
    
    def generate_initial_prices(self) -> Dict[int, Dict[int, float]]:
        """
        Generate initial prices for each day in the service horizon and each time period.
        
        Returns:
            Dictionary mapping time period to {day: price}
        """
        prices = {}
        for t in range(1, self.params.T + 1):
            prices[t] = {
                day: self.rng.uniform(self.params.price_min, self.params.price_max)
                for day in range(1, self.params.N + 1)
            }
        
        logger.info(f"Generated initial prices for {self.params.T} time periods")
        return prices
    
    def generate_study_instance(self, demand_scenario: str = 'base') -> Dict:
        """
        Generate a complete instance for the computational study.
        
        Returns:
            Dictionary containing all necessary parameters and data
        """
        return {
            'parameters': self.params,
            'booking_classes': self.booking_classes,
            'arrival_probabilities': self.generate_arrival_probabilities(demand_scenario),
            'reservation_price_params': self.generate_reservation_price_params(),
            'initial_prices': self.generate_initial_prices()
        }

In [9]:
def create_test_instance() -> Dict:
    """
    Create a small test instance for verification.
    
    Returns:
        Dictionary containing test instance data
    """
    # Create parameters for a small instance
    params = StudyParameters(
        T=14,  # 10 booking periods
        N=7,   # 5-day service horizon
        C=5,   # 5 rooms
        price_min=100.0,
        price_max=500.0,
        alpha=0.1,  # Smoothing parameter for price acceptance
        beta=0.1    # Smoothing parameter for capacity
    )
    
    # Generate test instance
    generator = DataGenerator(params, seed=42)
    test_instance = generator.generate_study_instance()
    
    logger.info("Created test instance")
    return test_instance

In [10]:
if __name__ == "__main__":
    # Generate and print a test instance
    test_instance = create_test_instance()
    
    # Print summary statistics
    print("\nTest Instance Summary:")
    print(f"Booking Horizon (T): {test_instance['parameters'].T}")
    print(f"Service Horizon (N): {test_instance['parameters'].N}")
    print(f"Capacity (C): {test_instance['parameters'].C}")
    print(f"Number of Booking Classes: {len(test_instance['booking_classes'])}")
    
    # Print example arrival probabilities for first time period
    print("\nExample Arrival Probabilities (t=1):")
    probs_t1 = test_instance['arrival_probabilities'][1]
    total_prob = sum(probs_t1.values())
    print(f"Total arrival probability: {total_prob:.3f}")
    
    # Print example reservation price parameters
    print("\nExample Reservation Price Parameters:")
    eps_params = test_instance['reservation_price_params']
    print(f"Average epsilon: {np.mean(list(eps_params.values())):.3f}")
    
    # Print example initial prices
    print("\nExample Initial Prices (t=1):")
    prices_t1 = test_instance['initial_prices'][1]
    print(f"Average price: {np.mean(list(prices_t1.values())):.2f}")

INFO:__main__:Generated 28 booking classes with max LOS = 7
INFO:__main__:Initialized DataGenerator with 28 booking classes
INFO:__main__:
Demand Scenario: base
INFO:__main__:Average daily arrival probability: 0.950
INFO:__main__:Maximum daily arrival probability: 0.950
INFO:__main__:Minimum daily arrival probability: 0.950
INFO:__main__:
Average arrival probabilities by day of week:
INFO:__main__:Sunday: 0.035
INFO:__main__:Monday: 0.028
INFO:__main__:Tuesday: 0.029
INFO:__main__:Wednesday: 0.031
INFO:__main__:Thursday: 0.038
INFO:__main__:Friday: 0.051
INFO:__main__:Saturday: 0.049
INFO:__main__:Generated reservation price parameters
INFO:__main__:Generated initial prices for 14 time periods
INFO:__main__:Created test instance



Test Instance Summary:
Booking Horizon (T): 14
Service Horizon (N): 7
Capacity (C): 5
Number of Booking Classes: 28

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Reservation Price Parameters:
Average epsilon: 0.001

Example Initial Prices (t=1):
Average price: 236.88


In [11]:
# Generate and print a test instance
test_instance = create_test_instance()

# Print summary statistics
print("\nTest Instance Summary:")
print(f"Booking Horizon (T): {test_instance['parameters'].T}")
print(f"Service Horizon (N): {test_instance['parameters'].N}")
print(f"Capacity (C): {test_instance['parameters'].C}")
print(f"Number of Booking Classes: {len(test_instance['booking_classes'])}")

# Print example arrival probabilities for first time period
print("\nExample Arrival Probabilities (t=1):")
probs_t1 = test_instance['arrival_probabilities'][3]
total_prob = sum(probs_t1.values())
print(f"Total arrival probability: {total_prob:.3f}")

# Print example reservation price parameters
print("\nExample Reservation Price Parameters:")
eps_params = test_instance['reservation_price_params']
print(f"Average epsilon: {np.mean(list(eps_params.values())):.3f}")

# Print example initial prices
print("\nExample Initial Prices (t=1):")
prices_t1 = test_instance['initial_prices'][1]
print(f"Average price: {np.mean(list(prices_t1.values())):.2f}")

INFO:__main__:Generated 28 booking classes with max LOS = 7
INFO:__main__:Initialized DataGenerator with 28 booking classes
INFO:__main__:
Demand Scenario: base
INFO:__main__:Average daily arrival probability: 0.950
INFO:__main__:Maximum daily arrival probability: 0.950
INFO:__main__:Minimum daily arrival probability: 0.950
INFO:__main__:
Average arrival probabilities by day of week:
INFO:__main__:Sunday: 0.035
INFO:__main__:Monday: 0.028
INFO:__main__:Tuesday: 0.029
INFO:__main__:Wednesday: 0.031
INFO:__main__:Thursday: 0.038
INFO:__main__:Friday: 0.051
INFO:__main__:Saturday: 0.049
INFO:__main__:Generated reservation price parameters
INFO:__main__:Generated initial prices for 14 time periods
INFO:__main__:Created test instance



Test Instance Summary:
Booking Horizon (T): 14
Service Horizon (N): 7
Capacity (C): 5
Number of Booking Classes: 28

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Reservation Price Parameters:
Average epsilon: 0.001

Example Initial Prices (t=1):
Average price: 236.88


In [12]:
booking_horizon = test_instance['parameters'].T
for i in range(1, booking_horizon+1):
    print("\nExample Arrival Probabilities (t=1):")
    probs_t1 = test_instance['arrival_probabilities'][i]
    total_prob = sum(probs_t1.values())
    print(f"Total arrival probability: {total_prob:.3f}")


Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950

Example Arrival Probabilities (t=1):
Total arrival probability: 0.950
